<i>Copyright (c) Microsoft Corporation.</i>

<i>Licensed under the MIT License.</i>

# ARIMA: Autoregressive Integrated Moving Average

This notebook provides an example of how to train an ARIMA model to generate point forecasts of product sales in retail. We will train an ARIMA based model on the Orange Juice dataset.

An ARIMA, which stands for AutoRegressive Integrated Moving Average, model can be created using an `ARIMA(p,d,q)` model within `statsmodels` library. In this notebook, we will be using an alternative library `pmdarima`, which allows us to automatically search for optimal ARIMA parameters, within a specified range. More specifically, we will be using `auto_arima` function within `pmdarima` to automatically discover the optimal parameters for an ARIMA model. This function wraps `ARIMA` and `SARIMAX` models of `statsmodels` library, that correspond to non-seasonal and seasonal model space, respectively.

In an ARIMA model there are 3 parameters that are used to help model the major aspects of a times series: seasonality, trend, and noise. These parameters are:
- **p** is the parameter associated with the auto-regressive aspect of the model, which incorporates past values.
- **d** is the parameter associated with the integrated part of the model, which effects the amount of differencing to apply to a time series.
- **q** is the parameter associated with the moving average part of the model.,

If our data has a seasonal component, we use a seasonal ARIMA model or `ARIMA(p,d,q)(P,D,Q)m`. In that case, we have an additional set of parameters: `P`, `D`, and `Q` which describe the autoregressive, differencing, and moving average terms for the seasonal part of the ARIMA model, and `m` refers to the number of periods in each season.


## Global Settings and Imports

In [1]:
import os
import sys
import math
import warnings
import itertools
import numpy as np
import pandas as pd
import scrapbook as sb

from pmdarima.arima import auto_arima

from fclib.common.utils import git_repo_path
from fclib.common.plot import plot_predictions_with_history
from fclib.evaluation.evaluation_utils import MAPE
from fclib.dataset.ojdata import download_ojdata, split_train_test, complete_and_fill_df

pd.options.display.float_format = "{:,.2f}".format
np.set_printoptions(precision=2)
warnings.filterwarnings("ignore")

print("System version: {}".format(sys.version))

System version: 3.6.10 |Anaconda, Inc.| (default, Jan  7 2020, 21:14:29) 
[GCC 7.3.0]


## Parameters

Next, we define global settings related to the model. We will use historical weekly sales data only, without any covariate features to train the ARIMA model. The model parameter ranges are provided in params. These are later used by the `auto_arima()` function to search the space for the optimal set of parameters. To increase the space of models to search over, increase the `max_p` and `max_q` parameters.

> NOTE: Our data does not show a strong seasonal component (as demonstrated in data exploration example notebook), so we will not be searching over the seasonal ARIMA models. To search over the seasonal models, set `seasonal` to `True` and include `start_P`, `start_Q`, `max_P`, and `max_Q` parameters in the auto_arima() function.

In [2]:
# Use False if you've already downloaded and split the data
DOWNLOAD_SPLIT_DATA = True

# Data directory
DATA_DIR = os.path.join(git_repo_path(), "ojdata")

# Forecasting settings
N_SPLITS = 5
HORIZON = 2
GAP = 2
FIRST_WEEK = 40
LAST_WEEK = 156

# Parameters of ARIMA model
params = {
    "seasonal": False,
    "start_p": 0,
    "start_q": 0,
    "max_p": 5,
    "max_q": 5,
    "m": 52,
}

## Data Preparation

We need to download the Orange Juice data and split it into training and test sets. By default, the following cell will download and spit the data. If you've already done so, you may skip this part by switching `DOWNLOAD_SPLIT_DATA` to `False`.

We store the training data and test data using dataframes. The training data includes `train_df` and `aux_df` with `train_df` containing the historical sales up to week 135 (the time we make forecasts) and `aux_df` containing price/promotion information up until week 138. Here we assume that future price and promotion information up to a certain number of weeks ahead is predetermined and known. In our example, we will be using historical sales only, and will not be using the `aux_df` data. The test data is stored in `test_df` which contains the sales of each product in week 137 and 138. Assuming the current week is week 135, our goal is to forecast the sales in week 137 and 138 using the training data. There is a one-week gap between the current week and the first target week of forecasting as we want to leave time for planning inventory in practice.

The setting of the forecast problem are defined in `fclib.dataset.ojdata.split_train_test` function. We can change this setting (e.g., modify the horizon of the forecast or the range of the historical data) by passing different parameters to this functions. Below, we split the data into `n_splits=N_SPLITS` splits, using the forecasting settings listed above in the **Parameters** section.

In [3]:
if DOWNLOAD_SPLIT_DATA:
    download_ojdata(DATA_DIR)
    train_df_list, test_df_list, _ = split_train_test(
        DATA_DIR,
        n_splits=N_SPLITS,
        horizon=HORIZON,
        gap=GAP,
        first_week=FIRST_WEEK,
        last_week=LAST_WEEK,
        write_csv=True,
    )

    print("Finished data downloading and splitting.")

Data already exists at the specified location.
Finished data downloading and splitting.


To create training data and test data for multi-round forecasting, we pass a number greater than `1` to `n_splits` parameter in `split_train_test()` function. Note that the forecasting periods we generate in each test round are **non-overlapping**. This allows us to evaluate the forecasting model on multiple rounds of data, and get a more robust estimate of our model's performance.

For visual demonstration, this is what the time series splits would look like for `N_SPLITS = 5`, and using other settings as above:

![Multi split](../../assets/time_series_split_multiround.jpg)


### Process training data

Our time series data is not complete, since we have missing sales for some stores/products and weeks. We will fill in those missing values by propagating the last valid observation forward to next available value.

Note that our time series are grouped by `store` and `brand`, while `week` represents a time step, and `logmove` represents the value to predict.

We will define functions for data frame processing, then use these functions within a loop that loops over each forecasting rounds.

In [4]:
def process_training_df(train_df):
    """Process training data frame."""
    train_df = train_df[["store", "brand", "week", "logmove"]]
    store_list = train_df["store"].unique()
    brand_list = train_df["brand"].unique()
    train_week_list = range(FIRST_WEEK, max(train_df.week))

    train_filled = complete_and_fill_df(train_df, stores=store_list, brands=brand_list, weeks=train_week_list)

    return train_filled

### Process test data

Let's now process the test data. Note that the test data runs from `LAST_WEEK - HORIZON + 1` to `LAST_WEEK`. Note that, in addition to filling out missing values, we also convert unit sales from logarithmic scale to the counts. We will do model training on the log scale, due to improved performance, however, we will transfrom the test data back into the unit scale (counts) by applying `math.exp()`, so that we can evaluate the performance on the unit scale.


In [5]:
def process_test_df(test_df):
    """Process test data frame."""
    test_df["actuals"] = test_df.logmove.apply(lambda x: round(math.exp(x)))
    test_df = test_df[["store", "brand", "week", "actuals"]]
    store_list = test_df["store"].unique()
    brand_list = test_df["brand"].unique()

    test_week_list = range(min(test_df.week), max(test_df.week) + 1)
    test_filled = complete_and_fill_df(test_df, stores=store_list, brands=brand_list, weeks=test_week_list)

    return test_filled

## Model training

Now let's run model training across all the stores and brands, and across all rounds. We will re-run the same code to automatically search for the best parameters, simply wrapped in a for loop iterating over stores and brands.

> NOTE: Since we are building a model for each time series sequentially, it will take ~11 minutes for a single round, to iterate over 900+ time series for each store and brand.

In [6]:
%%time
# Train and predict for all forecast rounds

# Create an empty df to store predictions
result_df = pd.DataFrame(None, columns=["predictions", "store", "brand", "week", "actuals", "round"])

for r in range(N_SPLITS):
    print("---------- Round " + str(r + 1) + " ----------")

    # Process training data set
    train_df = train_df_list[r].reset_index()
    train_filled = process_training_df(train_df)

    # Process test data set
    test_df = test_df_list[r].reset_index()
    test_filled = process_test_df(test_df)

    print("Training ARIMA model...")
    store_list = train_filled["store"].unique()
    brand_list = train_filled["brand"].unique()
    # store_list = store_list[0:10]
    for store, brand in itertools.product(store_list, brand_list):

        if brand == 1:
            print(f"Forecasting for store: {store}")

        train_ts = train_filled.loc[(train_filled.store == store) & (train_filled.brand == brand)]
        train_ts = np.array(train_ts["logmove"])

        model = auto_arima(
            train_ts,
            seasonal=params["seasonal"],
            start_p=params["start_p"],
            start_q=params["start_q"],
            max_p=params["max_p"],
            max_q=params["max_q"],
            stepwise=True,
        )

        model.fit(train_ts)
        preds = model.predict(n_periods=GAP + HORIZON - 1)
        predictions = np.round(np.exp(preds[-HORIZON:]))

        test_week_list = range(min(test_filled.week), max(test_filled.week) + 1)
        pred_df = pd.DataFrame(
            {"predictions": predictions, "store": store, "brand": brand, "week": test_week_list, "round": r + 1,}
        )
        test_ts = test_filled.loc[(test_filled.store == store) & (test_filled.brand == brand)]

        combined_df = pd.merge(pred_df, test_ts, on=["store", "brand", "week"], how="left")

        result_df = result_df.append(combined_df, ignore_index=True)

---------- Round 1 ----------
Training ARIMA model...
Forecasting for store: 2
Forecasting for store: 5
Forecasting for store: 8
Forecasting for store: 9
Forecasting for store: 12
Forecasting for store: 14
Forecasting for store: 18
Forecasting for store: 21
Forecasting for store: 28
Forecasting for store: 32
Forecasting for store: 33
Forecasting for store: 40
Forecasting for store: 44
Forecasting for store: 45
Forecasting for store: 47
Forecasting for store: 48
Forecasting for store: 49
Forecasting for store: 50
Forecasting for store: 51
Forecasting for store: 52
Forecasting for store: 53
Forecasting for store: 54
Forecasting for store: 56
Forecasting for store: 59
Forecasting for store: 62
Forecasting for store: 64
Forecasting for store: 67
Forecasting for store: 68
Forecasting for store: 70
Forecasting for store: 71
Forecasting for store: 72
Forecasting for store: 73
Forecasting for store: 74
Forecasting for store: 75
Forecasting for store: 76
Forecasting for store: 77
Forecasting fo

Forecasting for store: 113
Forecasting for store: 114
Forecasting for store: 115
Forecasting for store: 116
Forecasting for store: 117
Forecasting for store: 118
Forecasting for store: 119
Forecasting for store: 121
Forecasting for store: 122
Forecasting for store: 123
Forecasting for store: 124
Forecasting for store: 126
Forecasting for store: 128
Forecasting for store: 129
Forecasting for store: 130
Forecasting for store: 131
Forecasting for store: 132
Forecasting for store: 134
Forecasting for store: 137
---------- Round 5 ----------
Training ARIMA model...
Forecasting for store: 2
Forecasting for store: 5
Forecasting for store: 8
Forecasting for store: 9
Forecasting for store: 12
Forecasting for store: 14
Forecasting for store: 18
Forecasting for store: 21
Forecasting for store: 28
Forecasting for store: 32
Forecasting for store: 33
Forecasting for store: 40
Forecasting for store: 44
Forecasting for store: 45
Forecasting for store: 47
Forecasting for store: 48
Forecasting for store

> Note: Since `auto_arima` model makes consecutive forecasts from the last time point, we want to forecast the next `n_periods = GAP + HORIZON - 1` points, so that we can account for the GAP, as described in the data setup.

## Model evaluation

To evaluate the model, we will use *mean absolute percentage error* or **MAPE**.

In [ ]:
metric_value = MAPE(result_df.predictions, result_df.actuals) * 100
sb.glue("MAPE", metric_value)

print(f"MAPE of the forecasts across all rounds is {metric_value} %")

Let's take a look at the `MAPE` for each round.

In [ ]:
mape_r = result_df.groupby("round").apply(lambda x: MAPE(x.predictions, x.actuals) * 100)

print("MAPE values for each forecasting round are:")
print(mape_r)

When building a model with `auto_arima` for a large number of time series, it is often difficult to examine each model individually (in a similar way we did for the single time series above). As `auto_arima` searches a restricted space of the models, defined by the range of `p` and `q` parameters, we often might not find an optimal model for each time series.

Let's plot a few examples of forecasted results from the last round.

In [ ]:
num_samples = 6
min_week = 120
sales = pd.read_csv(os.path.join(DATA_DIR, "yx.csv"))
sales["move"] = sales.logmove.apply(lambda x: round(math.exp(x)) if x > 0 else 0)

result_df = result_df[result_df["round"] == 1]

result_df["move"] = result_df.predictions
plot_predictions_with_history(
    result_df,
    sales,
    grain1_unique_vals=store_list,
    grain2_unique_vals=brand_list,
    time_col_name="week",
    target_col_name="move",
    grain1_name="store",
    grain2_name="brand",
    min_timestep=min_week,
    num_samples=num_samples,
    predict_at_timestep=145,
    line_at_predict_time=True,
    title="Prediction results for a few sample time series (predictions are made at week 145)",
    x_label="week",
    y_label="unit sales",
    random_seed=2,
)